# Import Library 

In [143]:
# load dependencies'
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from shapely.geometry import Point, LineString, Polygon
import pyproj 
import mm_utils
from FIS2 import FIS2
from datetime import datetime

In [144]:
def get_bearing(point1, point2):
    # this code calculate the bearing of any given pair of longitude, latitude  
    geodesic = pyproj.Geod(ellps='WGS84')
    fwd_azimuth,back_azimuth,distance = geodesic.inv(point1[0], point1[1], point2[0], point2[1])
    return fwd_azimuth

def edge_bearing(edge):
    # this function calculate the bearing from the starting and ending node of each road segment
    bearing = get_bearing(edge[0], edge[len(edge) - 1])
    return(bearing)
    
def conv_angle(angle):
    # this function convert angle from -pi,pi to 0,2*pi
    if angle < 0 :
        angle = angle + 360
    return(angle)

def conc(a):
    #function to convert list or integer in osmid into a unique string id 
    if type(a) is int:
        return str(a)
    ans = ",".join(map(str, a))
    return ans

def err_polygon(curr_loc, err_size):
    # function that output shapely polygon for point error bound
    x = curr_loc['geometry'].iloc[0].x
    y = curr_loc['geometry'].iloc[0].y
    
    err_coord = [[x - err_size, y + err_size], 
                 [x + err_size, y + err_size],
                 [x + err_size, y - err_size],
                 [x - err_size, y - err_size]]

    poly_coord = Polygon(err_coord)
    # #print(ply_coord)
    df = {'Attribute' : ['name1'], 'geometry':poly_coord}

    #projected to UTM 31 
    err_poly = gpd.GeoDataFrame(df, geometry = 'geometry', crs = "EPSG:32631")
    
    return err_poly

In [166]:
gdf_utm = pd.read_pickle('gdf_utm.pkl')
edges_utm = pd.read_pickle('edges_utm.pkl')
nodes_utm = pd.read_pickle('nodes_utm.pkl')
curr_edge = pd.read_pickle('current_edge.pkl')

# for debugging purposes start at iteration point 15 
point_index = 13
# this is for debugging purposes, true 
last_matched = gpd.GeoDataFrame(geometry=gpd.points_from_xy([819609.861], [5766244.800]), crs="EPSG:32631")

# current location 
curr_loc = gdf_utm.iloc[point_index].to_frame().T

# find longitude and latitude for last matched data
last_matched['lon_lat'] = last_matched.to_crs({'init': 'epsg:4326'})


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [175]:
# extract node id
edge_node1 = nodes_utm.loc[curr_edge.index[0][0]]
edge_node2 = nodes_utm.loc[curr_edge.index[0][1]]

# select the end node 
if (curr_loc['GPS Bearing'].iloc[0] > 0 and curr_loc['GPS Bearing'].iloc[0] < 180):
    # if object movint to the right of map select the one with larger longitude
    if edge_node1.x > edge_node2.x:
        end_node = edge_node1
        start_node = edge_node2
    else:
        end_node = edge_node2
        start_node = edge_node1
else:
    # if object is moving to the left select node with smaller longitude
    if edge_node1.x < edge_node2.x:
        end_node = edge_node1
        start_node = edge_node2
    else:
        end_node = edge_node2
        start_node = edge_node1


# finding angle alpha, which is the angle between previous matched point and current postion 
# find pos_bearing1, bearing between last matched point and current point
# convert two points into tuple for function input purposes
last_matched_tuple = (last_matched['lon_lat'].x.iloc[0], last_matched['lon_lat'].y.iloc[0])
cur_loc_tuple = curr_loc['lon_lat'].iloc[0].x, curr_loc['lon_lat'].iloc[0].y
pos_bearing1 = get_bearing(last_matched_tuple, cur_loc_tuple)
# convert to 0, 2pi 
pos_bearing1 = conv_angle(pos_bearing1)

# find bearing of the edge 
# convert lat lon into tupple coordinate 
start_node_tuple = (start_node.x, start_node.y)
end_node_tuple = (end_node.x, end_node.y)
# bearing between start to end
start_bearing = get_bearing(start_node_tuple, end_node_tuple)
# convert to 0 ,2*pi
start_bearing = conv_angle(start_bearing)
alpha = abs(pos_bearing - start_bearing)

# find beta, angle between end node and current position 
# bearing from end to start 
end_bearing = get_bearing(end_node_tuple, start_node_tuple)
end_bearing = conv_angle(end_bearing)
#pos_bearing2 beating between end node and current position 
pos_bearing2 = get_bearing(end_node_tuple, cur_loc_tuple)
pos_bearing = conv_angle(pos_bearing)
# beta equal to abs(end_bearing minus pos_bearing2)
beta = abs(end_bearing - pos_bearing)


# Heading Increment  
HI = gdf_utm['GPS Bearing'].iloc[point_index] - gdf_utm['GPS Bearing'].iloc[point_index - 1]

# distance travelled from last position fix to the end nodes
d = end_node['geometry'].distance(curr_loc['geometry']).iloc[0]

# distance travelled since last position fix 
t = gdf_utm['time'].iloc[point_index] - gdf_utm['time'].iloc[point_index - 1]
d2 = (gdf_utm['speed_mps'].iloc[point_index])* t.seconds

delta_d = d - d2

speed = gdf_utm['speed_mps'][point_index]
hdop = gdf_utm['GPS HDOP'][point_index]

# rearrange new data to the input of fis1  
new_data = np.array([speed,hdop, alpha, beta, delta_d, HI, HI]).T

FIS2(new_data, plot = False)


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


49.99819310019818